In [3]:
## lfm api hack
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime, timedelta

championship_tracks = [
    "Spielberg - Red Bull Ring",
    "Hungaroring",
    "Circuit de Catalunya",
    "Snetterton","Nürburgring",
    "Autodromo Enzo e Dino Ferrari",
    "Silverstone",
    ]

def get_track_ids():
    response = requests.get("https://api3.lowfuelmotorsport.com/api/hotlaps/tracks")
    response_content = response.json()
    track_id_list = {}
    for item in response_content:
        track_id_list.update({item["track_name"]: item["track_id"]})
    return track_id_list

def get_hotlaps_from_every_track(championship_tracks,track_id_list):
    season_hotlap_collection = []
    for track in championship_tracks:
        response = requests.get(f"https://api3.lowfuelmotorsport.com/api/hotlaps?track={track_id_list[track]}&class=&car=&major=1.10&version=1.9.6&source=")
        response_content = response.json()
        season_hotlap_collection.append(response_content)
        print(response)
    return season_hotlap_collection

def get_all_cars_in_top_200(hotlap_collection, car_class: str):
    cars = []
    for item in hotlap_collection[0]["data"]:
        if item["car_name"] not in cars and item["car_class"] == car_class:
            cars.append(item["car_name"])
    return cars

def collect_laptimes_for_all_car(cars: list, hotlap_collection,championship_tracks): 
    collecton_for_all_track = []
    for car in cars:
        cars_and_times = {}
        cars_and_times[car] = []
        for i in range(0,len(championship_tracks)):
            counter = 0
            for times in hotlap_collection[i]["data"]:
                if times["car_name"] == car and counter != 5:
                    counter = counter + 1
                    cars_and_times[car].append(times["lap"])
        cars_and_times[car] = cars_and_times[car]
        collecton_for_all_track.append(cars_and_times)
    return collecton_for_all_track


def avg_the_collected_laptimes(collected_laptimes):
    collected_laptimes_avg = {}
    for items_all in collected_laptimes:
        for key, value in items_all.items():
            number_of_laptimes = len(value)
            full_sum_of_laptimes = timedelta()
            for times in value:
                td = datetime.strptime(times, "%M:%S.%f")
                td = timedelta(minutes=td.minute,seconds=td.second,milliseconds=td.microsecond/1000)
                full_sum_of_laptimes = full_sum_of_laptimes + td
            full_sum_of_laptimes = full_sum_of_laptimes/number_of_laptimes
            collected_laptimes_avg[key] = full_sum_of_laptimes
    return collected_laptimes_avg


track_id_list = get_track_ids()
hotlap_collection = get_hotlaps_from_every_track(championship_tracks, track_id_list)
cars = get_all_cars_in_top_200(hotlap_collection, "GT3")
collected_laptimes = collect_laptimes_for_all_car(cars,hotlap_collection,championship_tracks)
collected_laptimes_avg = avg_the_collected_laptimes(collected_laptimes)
sorted(collected_laptimes_avg)
print("Avarage laptime ")
sorted_x = sorted(collected_laptimes_avg.items(), key=lambda kv: kv[1])
df = pd.DataFrame(sorted_x, columns=['Car', 'Avg'])
df

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Avarage laptime 


,Car,Avg
0,Bentley Continental,0 days 00:01:42.772310
1,Ford Mustang GT3,0 days 00:01:43.275343
2,Ferrari 296 GT3,0 days 00:01:43.401943
3,BMW M4 GT3,0 days 00:01:43.412800
4,Lexus RC F GT3,0 days 00:01:43.452909
5,McLaren 720S GT3 Evo,0 days 00:01:43.538229
6,AMR V8 Vantage,0 days 00:01:43.561771
7,Porsche 992 GT3 R,0 days 00:01:43.679171
8,Nissan GT-R Nismo GT3,0 days 00:01:43.817114
9,Audi R8 LMS GT3 evo II,0 days 00:01:43.889794
